<a href="https://colab.research.google.com/github/MajstorKatastrofe/AI_NM1.1/blob/main/CNN_mojaArhitektura_PyTotch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

In [ ]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        self.img_dir = img_dir
        self.transform = transform
        self.img_labels = self._get_image_paths_and_labels()

    def _get_image_paths_and_labels(self):
        img_labels = []
        for root, dirs, files in os.walk(self.img_dir):
            for file in files:
                if file.endswith('.jpg') and 'models' not in root and '.ipynb_checkpoints' not in root:
                    label = root.split('/')[-1]  # Pretpostavlja se da je ime foldera labela
                    img_labels.append((os.path.join(root, file), label))
        return img_labels

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path, label = self.img_labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = 'C:\Users\pc\Documents\fashion-mnist-master\data\fashion'
dataset = CustomImageDataset(img_dir=dataset_path, transform=transform)
loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 7)

    def forward(self, x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.max_pool2d(x, 2)
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [ ]:
model = MyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Mapiranje tekstualnih labela u numeričke vrednosti
label_to_index = {
    'Do': 0, 'Re': 1, 'Mi': 2, 'Fa': 3, 'Sol': 4, 'La': 5, 'Si': 6
}

# Dodajte transformaciju labela u DataLoader loop
for epoch in range(10):
    for images, labels in loader:
        labels = torch.tensor([label_to_index[label] for label in labels])  # Pretvaranje labela u tenzore
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 2.0309085845947266
Epoch 2, Loss: 1.946516752243042
Epoch 3, Loss: 1.955647349357605
Epoch 4, Loss: 1.942591905593872
Epoch 5, Loss: 1.926315188407898
Epoch 6, Loss: 1.9334315061569214
Epoch 7, Loss: 1.9579828977584839
Epoch 8, Loss: 1.9013144969940186
Epoch 9, Loss: 1.4928736686706543
Epoch 10, Loss: 1.5094728469848633


In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# Putanja do slike koju testiramo
image_path = '/content/drive/MyDrive/NeuronskeMreze/Sol/sol15.jpg'

# Transformacije koje treba primeniti na sliku
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Ista veličina kao za trening set
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Iste normalizacione vrednosti
])

# Učitavanje i transformacija slike
image = Image.open(image_path).convert('RGB')
image = transform(image)
image = image.unsqueeze(0)

In [ ]:
model.eval()

with torch.no_grad():  # Isključivanje izračunavanja gradijenta za predikciju
    outputs = model(image)
    _, predicted = torch.max(outputs, 1)  # Dobijamo indeks najvećeg izlaza

# Mapiranje indeksa nazad u labele
index_to_label = {v: k for k, v in label_to_index.items()}
predicted_label = index_to_label[predicted.item()]

print(f'Predikcija modela: {predicted_label}')

Predikcija modela: Mi


In [ ]:
#file_path = "C:\\Users\\pc\\HuggFace"

file_path = "model.pth"
torch.save(model.state_dict(), file_path)

In [ ]:
current_directory = os.getcwd()
print("Current working directory:", current_directory)

Current working directory: /content


In [ ]:
# Get the current working directory
current_directory = os.getcwd()

# Construct the full path to the saved file
file_path = os.path.join(current_directory, "model.pth")

# Check if the file exists
if os.path.exists(file_path):
    print("File found at:", file_path)
else:
    print("File not found.")


File found at: /content/model.pth


In [ ]:
from google.colab import files

# Specify the file path
file_path = "/content/model.pth"

# Download the file
files.download(file_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>